In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
#Export
from uti.interface import *

In [3]:
path = Path('/home/jupyter/insight_project/Project-M/data/preprocessed/')
path.ls()

[PosixPath('/home/jupyter/insight_project/Project-M/data/preprocessed/ed91c398-31c6-437f-a9d1-462e3ccfb6fa.json'),
 PosixPath('/home/jupyter/insight_project/Project-M/data/preprocessed/example.txt'),
 PosixPath('/home/jupyter/insight_project/Project-M/data/preprocessed/ff063ea9-62b8-4f29-9faa-04a09cb5fba2.json'),
 PosixPath('/home/jupyter/insight_project/Project-M/data/preprocessed/.ipynb_checkpoints'),
 PosixPath('/home/jupyter/insight_project/Project-M/data/preprocessed/delete_test'),
 PosixPath('/home/jupyter/insight_project/Project-M/data/preprocessed/fabab216-0767-4aa5-85fa-bb8852eb30d3.json'),
 PosixPath('/home/jupyter/insight_project/Project-M/data/preprocessed/csv'),
 PosixPath('/home/jupyter/insight_project/Project-M/data/preprocessed/f6d2081a-0f79-4abf-9021-c4d254859890.json')]

In [4]:
test = Interface(path.ls(),eval_mode=True)
test.path

[PosixPath('/home/jupyter/insight_project/Project-M/data/preprocessed/ed91c398-31c6-437f-a9d1-462e3ccfb6fa.json'),
 PosixPath('/home/jupyter/insight_project/Project-M/data/preprocessed/ff063ea9-62b8-4f29-9faa-04a09cb5fba2.json'),
 PosixPath('/home/jupyter/insight_project/Project-M/data/preprocessed/fabab216-0767-4aa5-85fa-bb8852eb30d3.json'),
 PosixPath('/home/jupyter/insight_project/Project-M/data/preprocessed/f6d2081a-0f79-4abf-9021-c4d254859890.json')]

In [5]:
test.pre_processing(test=True,lm_path='/home/jupyter/insight_project/Project-M/data/preprocessed/csv/models')

In [6]:
len(test.data_list),test.eval_mode

(4, True)

In [7]:
#Export
class Trainer():
    '''Take interface, which is preprocessed data
        create learner base on the data
        save the model in the dest
    '''
    def __init__(self,data,pre_trained=True,model_dir=None):
        self.interface = data
        self.pre_trained = pre_trained
        if not os.path.exists(self.interface.csv_path/'models'):
            os.makedirs(self.interface.csv_path/'models')
        self.dest = self.interface.csv_path/'models'
        #hard code path, get url when host
        #fix
        #################
        self.url = '/home/jupyter/insight_project/Project-M/data/preprocessed/csv/models/general-clasifier-0.84'
        #################
        
    def _create_leaner(self,one_data):
        learn = text_classifier_learner(one_data,AWD_LSTM,drop_mult=0.5,
                                loss_func=FlattenedLoss(LabelSmoothingCrossEntropy),
                                metrics=[accuracy], #FBeta(beta=1)
                                model_dir = self.dest
                               )
        if torch.cuda.is_available():
            learn.to_fp16()
        learn.load(self.url)
        learn.freeze_to(-1)
        return learn
    
    def train_individual_clasifier(self):
        num_models = len(self.interface.data_list)
        counter_to_print = 0
        print('Creating models... total job =',num_models)
        for data in self.interface.data_list:
            print(f'Creating model #{counter_to_print+1}...')
            learn = self._create_leaner(data)
            learn.fit_one_cycle(1,1e-1,moms=(0.8,0.7))
            current_valid_score = learn.recorder.metrics[0][0].item()
            model_path = learn.save(f'{self.interface.dataset_name[counter_to_print]}_{current_valid_score:.2f}',
                                    return_path=True, with_opt=False)
            print(f'Model saved at {model_path}')
            counter_to_print += 1
#             print(f'Saving model to {self.dest}')
#             print(f'model name: {self.interface.dataset_name[counter_to_print]}_{current_valid_score}')
        

In [8]:
test_train = Trainer(test)

In [9]:
test_train.interface.dataset_name

['ed91c398-31c6-437f-a9d1-462e3ccfb6fa',
 'fabab216-0767-4aa5-85fa-bb8852eb30d3',
 'ff063ea9-62b8-4f29-9faa-04a09cb5fba2',
 'f6d2081a-0f79-4abf-9021-c4d254859890']

In [10]:
test_train.train_individual_clasifier()

Creating models... total job = 4
Creating model #1...


epoch,train_loss,valid_loss,accuracy,time
0,0.492518,0.502265,0.823529,00:07


Model saved at /home/jupyter/insight_project/Project-M/data/preprocessed/delete_test/models/ed91c398-31c6-437f-a9d1-462e3ccfb6fa_0.82.pth
Creating model #2...


epoch,train_loss,valid_loss,accuracy,time
0,0.519696,0.504885,0.793814,00:01


Model saved at /home/jupyter/insight_project/Project-M/data/preprocessed/delete_test/models/fabab216-0767-4aa5-85fa-bb8852eb30d3_0.79.pth
Creating model #3...


epoch,train_loss,valid_loss,accuracy,time
0,0.729507,0.509135,0.821918,00:01


Model saved at /home/jupyter/insight_project/Project-M/data/preprocessed/delete_test/models/ff063ea9-62b8-4f29-9faa-04a09cb5fba2_0.82.pth
Creating model #4...


epoch,train_loss,valid_loss,accuracy,time
0,0.425655,0.439271,0.851695,00:02


Model saved at /home/jupyter/insight_project/Project-M/data/preprocessed/delete_test/models/f6d2081a-0f79-4abf-9021-c4d254859890_0.85.pth


In [28]:
a = 0.873123
print(f'a is {a:.2f}')

a is 0.87


In [23]:
test.dest

PosixPath('/home/jupyter/insight_project/Project-M/data/preprocessed/delete_test/models')

In [16]:
!ls /home/jupyter/insight_project/Project-M/data/preprocessed/csv/models

1a.pth			    fine-tuned-enc-fwd.pth
awd_lstm_baseline-0917.pth  fine-tuned-fwd.pth
data_lm_large_bwd.pkl	    general-clasifier-0.84.pth
data_lm_large.pkl	    general-clasifier-bwd-0.83.pth
data_lm.pkl		    models
fine-tuned-bwd.pth	    textlist_class_seed42
fine-tuned-enc-bwd.pth	    tmp.pth


In [1]:
from notebook2script import *

In [2]:
notebook2script('Test_trainer.ipynb','trainer')

Converted Test_trainer.ipynb to uti/trainer.py
